Author: Kevin ALBERT  

Created: May 2020 

# Dashboard
_**how to deploy a Dash app to Azure App Service in the cloud using containers**_

## Contents
1. [Setup](#Setup)
1. [Develop](#Develop)
1. [Verify](#Verify)
1. [Public](#Public)
1. [Private](#Private)
1. [Cleanup](#Cleanup)

# Setup

* required
  * open a ssh putty session to this VM
  * open a cloud powershell session
* optional
  * [register](https://hub.docker.com/) docker hub account

# Develop

Development files and directory for containerization.  
Asuming the dashboard python code already developed using VSCode and debugger.

1. parameters
1. project folder
1. project files
  * requirements.txt
  * application.py
  * Dockerfile

### parameters

In [2]:
IPADDR           = "51.136.61.6" # this VM IP
PROJNAME         = "dash-azure" # docker image name and project folder name
DOCKERHUB_USER   = "beire2018" # docker hub account name
AZUREDOCKER_USER = "container2804206" # azure docker container registry account name
DOCKER_TAG       = "latest" # {v1.0.0, latest}
WEBAPP_PORT      = "80" # {default 80, 8050}

In [3]:
RESOURCE_GROUP                = "myResourceGroup6"
LOCATION                      = "northeurope" # {westeurope, northeurope, eastus}
APPSERVICEPLAN_NAME           = "myAppServicePlan6"
APPSERVICEPLAN_COMPUTE        = "F1" # {F1 free 1G, B1 €11/m 1.7G, B2 3.5G, B3 7G}=DEV, {P1V2}=PROD 
WEBAPP_NAME                   = "kevin2404206"
DOCKER_HUB_CONTAINER_NAME     = DOCKERHUB_USER + "/" + PROJNAME + ":" + DOCKER_TAG # <username>/<container-or-image>:<tag>
AZURE_CONTAINER_REGISTRY_NAME = AZUREDOCKER_USER + ".azurecr.io/" + PROJNAME + ":" + DOCKER_TAG

### project folder

In [4]:
import os
project_folder = '../dashboards/' + PROJNAME
os.makedirs(project_folder, exist_ok=True)
print(PROJNAME, 'folder created')

dash-azure folder created


### project files

**requirements.txt**  
python packages list to be installed in container

In [5]:
%%writefile $project_folder/requirements.txt
dash

Writing ../dashboards/dash-azure/requirements.txt


**application.py**  
plotly dash code to run a flask service in python (port: 80)

In [6]:
%%writefile $project_folder/application.py
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
application = app.server

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''This is Dash running on Azure App Service.'''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

if __name__ == '__main__':
    application.run(debug=True, host='0.0.0.0', port='80')  # use port 80 for Azure Web App service

Writing ../dashboards/dash-azure/application.py


**Dockerfile**  
docker container build steps, open an interactive shell to develop steps and remove upon exit
```sh
sudo docker container run -p 8050:80 --rm -it python:3.6-alpine /bin/sh
    pip -V
    mkdir /app
    cd /app
    vi requirements.txt
    pip install -r requirements.txt
    vi application.py
    python application.py
```

In [7]:
%%writefile $project_folder/Dockerfile
FROM python:3.6-slim

RUN mkdir /app
WORKDIR /app
ADD . /app/
RUN pip install -r requirements.txt

ENTRYPOINT [ "python" ]
CMD ["application.py"]

Writing ../dashboards/dash-azure/Dockerfile


# Verify

Test run the docker container on the cloud VM (local)

1. build image
1. run image

### build image

In [8]:
print("sudo docker build -t " + PROJNAME + " " + project_folder +"/.")

sudo docker build -t dash-azure ../dashboards/dash-azure/.


In [9]:
! sudo docker build -t $PROJNAME $project_folder/.

Sending build context to Docker daemon  4.608kB
Step 1/8 : FROM python:3.6-alpine
3.6-alpine: Pulling from library/python
Digest: sha256:50c53d4419a5049e35009691c43d3dbf0cb6c29c14be25965af480ba77a8367f
Status: Downloaded newer image for python:3.6-alpine
 ---> 2bb6435d63d9
Step 2/8 : RUN mkdir /app
 ---> Using cache
 ---> feb9dd301a9b
Step 3/8 : WORKDIR /app
 ---> Using cache
 ---> 9a898320a08e
Step 4/8 : ADD . /app/
 ---> 65846955f534
Step 5/8 : RUN apk add g++
 ---> Running in 625a2abef457
fetch http://dl-cdn.alpinelinux.org/alpine/v3.11/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.11/community/x86_64/APKINDEX.tar.gz
(1/13) Installing libgcc (9.2.0-r4)
(2/13) Installing libstdc++ (9.2.0-r4)
(3/13) Installing binutils (2.33.1-r0)
(4/13) Installing gmp (6.1.2-r1)
(5/13) Installing isl (0.18-r0)
(6/13) Installing libgomp (9.2.0-r4)
(7/13) Installing libatomic (9.2.0-r4)
(8/13) Installing mpfr4 (4.0.2-r1)
(9/13) Installing mpc1 (1.1.0-r1)
(10/13) Installing g

### run image

In [10]:
print("===>  http://" + IPADDR + ":8050/", "\n")
! sudo docker run -it --rm -p 8050:80 $PROJNAME   # bridge application.py port 80 to external port 8050

    # stop cell when done to continue...

===>  http://51.136.61.6:8050/ 

 * Serving Flask app "application" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 834-696-868
84.198.32.146 - - [28/Apr/2020 06:33:57] "GET / HTTP/1.1" 200 -
84.198.32.146 - - [28/Apr/2020 06:33:58] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_3_0m1588055601.8.7.min.js HTTP/1.1" 200 -
84.198.32.146 - - [28/Apr/2020 06:33:58] "GET /_dash-component-suites/dash_renderer/react@16.v1_3_0m1588055601.13.0.min.js HTTP/1.1" 200 -
84.198.32.146 - - [28/Apr/2020 06:33:58] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_3_0m1588055601.13.0.min.js HTTP/1.1" 200 -
84.198.32.146 - - [28/Apr/2020 06:33:58] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_3_0m1588055601.7.2.min.js HTTP/1.1" 200 -
84.198.32.146 - - [28/Apr/2020 06:33:58] "GET /_dash-component-suites/

# Public
**option 1:** login, build and deploy the web app using **docker hub**

1. login
1. build image
1. push image
1. deploy web app

### login
open a putty session and execute this command

In [11]:
# login to your personal docker hub account
print("sudo docker login --username " + DOCKERHUB_USER)

sudo docker login --username beire2018


### build image

In [12]:
print("sudo docker build -t " + DOCKERHUB_USER + "/" + PROJNAME + " " + project_folder +"/.")

sudo docker build -t beire2018/dash-azure ../dashboards/dash-azure/.


In [13]:
! sudo docker build -t $DOCKERHUB_USER/$PROJNAME $project_folder/.

Sending build context to Docker daemon  4.608kB
Step 1/8 : FROM python:3.6-alpine
 ---> 2bb6435d63d9
Step 2/8 : RUN mkdir /app
 ---> Using cache
 ---> feb9dd301a9b
Step 3/8 : WORKDIR /app
 ---> Using cache
 ---> 9a898320a08e
Step 4/8 : ADD . /app/
 ---> Using cache
 ---> 65846955f534
Step 5/8 : RUN apk add g++
 ---> Using cache
 ---> eda5da76a925
Step 6/8 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> d76fdbab75c9
Step 7/8 : ENTRYPOINT [ "python" ]
 ---> Using cache
 ---> 481d3f2576f9
Step 8/8 : CMD ["application.py"]
 ---> Using cache
 ---> 7d39d3496fe1
Successfully built 7d39d3496fe1
Successfully tagged beire2018/dash-azure:latest


### push image

In [14]:
print("sudo docker push " + DOCKERHUB_USER + "/" + PROJNAME)

sudo docker push beire2018/dash-azure


In [15]:
# upload the container image
! sudo docker push $DOCKERHUB_USER/$PROJNAME

The push refers to repository [docker.io/beire2018/dash-azure]

de9733a8: Preparing 
8c13a0f3: Preparing 
2e4d5b30: Preparing 
2c0b805b: Preparing 
a45a13cc: Preparing 
9655bf6d: Preparing 
6063a67d: Preparing 
b76feca4: Preparing 
8c13a0f3: Pushed     172MB/170.6MBAlatest: digest: sha256:6733c907d50bc8b40d50b980b0dc50a8ae9d1fac3d42cc7c7ec5aed82ca1ed98 size: 2205


### deploy web app
update the parameters and run the cell to generate a deployment script  
connect to cloud powershell, upload script, make executable and run

In [16]:
# parameters
print('RESOURCE_GROUP="' + RESOURCE_GROUP + '"')
print('LOCATION="' + LOCATION + '"')
print('APPSERVICEPLAN_NAME="' + APPSERVICEPLAN_NAME + '"')
print('APPSERVICEPLAN_COMPUTE="' + APPSERVICEPLAN_COMPUTE + '"')
print('WEBAPP_NAME="' + WEBAPP_NAME + '"')
print('DOCKER_HUB_CONTAINER_NAME="' + DOCKER_HUB_CONTAINER_NAME + '"')

RESOURCE_GROUP="myResourceGroup6"
LOCATION="northeurope"
APPSERVICEPLAN_NAME="myAppServicePlan6"
APPSERVICEPLAN_COMPUTE="B1"
WEBAPP_NAME="kevin2404206"
DOCKER_HUB_CONTAINER_NAME="beire2018/dash-azure:latest"


In [17]:
%%writefile $project_folder/publicDashboard_deploy.azcli
#!/bin/bash

# you will need to login first
az login

# update these parameters
RESOURCE_GROUP="myResourceGroup6"
LOCATION="northeurope"
APPSERVICEPLAN_NAME="myAppServicePlan6"
APPSERVICEPLAN_COMPUTE="B1"
WEBAPP_NAME="kevin2404206"
DOCKER_HUB_CONTAINER_NAME="beire2018/dash-azure:latest"

# create a resource group
az group create --name $RESOURCE_GROUP --location $LOCATION

# create an app service plan
az appservice plan create \
  --name $APPSERVICEPLAN_NAME \
  --resource-group $RESOURCE_GROUP \
  --sku $APPSERVICEPLAN_COMPUTE \
  --is-linux \
  --location $LOCATION

# create a web app with Docker Hub
az webapp create \
  --resource-group $RESOURCE_GROUP \
  --plan $APPSERVICEPLAN_NAME \
  --name $WEBAPP_NAME \
  --deployment-container-image-name $DOCKER_HUB_CONTAINER_NAME

# web app URL
echo "services loading... (~2min)"
echo "http://${WEBAPP_NAME}.azurewebsites.net"

Writing ../dashboards/dash-azure/publicDashboard_deploy.azcli


# Private
**option 2:** login, build and deploy the web app using **azure container registry**  

1. create azure container registry
1. login 
1. build image
1. push image
1. deploy web app

### create azure container registry
update the parameters and run the cell to generate a deployment script  
connect to cloud powershell, upload script, make executable and run

In [18]:
# parameters
print('RESOURCE_GROUP="' + RESOURCE_GROUP + '"')
print('LOCATION="' + LOCATION + '"')
print('CONTAINER_NAME="' + AZUREDOCKER_USER + '"')

RESOURCE_GROUP="myResourceGroup6"
LOCATION="northeurope"
CONTAINER_NAME="container2804206"


In [19]:
%%writefile $project_folder/containerRegistry_deploy.azcli
#!/bin/bash

# you will need to login first
az login

# update these parameters
RESOURCE_GROUP="myResourceGroup6"
LOCATION="northeurope"
CONTAINER_NAME="container2804206"

# Create an Azure Resource group
az group create --name $RESOURCE_GROUP --location $LOCATION
# Create an Azure Container Registry
az acr create --name $CONTAINER_NAME --resource-group $RESOURCE_GROUP --sku Basic --admin-enabled true
# Retrieve Azure Container Registry Credentials
az acr credential show --name $CONTAINER_NAME

Writing ../dashboards/dash-azure/containerRegistry_deploy.azcli


### login
open a putty session and execute this command

In [20]:
# login to your private azure container registry account
print("sudo docker login " + AZUREDOCKER_USER + ".azurecr.io --username " + AZUREDOCKER_USER)

sudo docker login container2804206.azurecr.io --username container2804206


### build image

In [21]:
print("sudo docker build -t " + AZUREDOCKER_USER + ".azurecr.io/" + PROJNAME + " " + project_folder +"/.")

sudo docker build -t container2804206.azurecr.io/dash-azure ../dashboards/dash-azure/.


In [22]:
! sudo docker build -t $AZUREDOCKER_USER'.azurecr.io/'$PROJNAME $project_folder/.

Sending build context to Docker daemon  7.168kB
Step 1/8 : FROM python:3.6-alpine
 ---> 2bb6435d63d9
Step 2/8 : RUN mkdir /app
 ---> Using cache
 ---> feb9dd301a9b
Step 3/8 : WORKDIR /app
 ---> Using cache
 ---> 9a898320a08e
Step 4/8 : ADD . /app/
 ---> 54bc44a9e72e
Step 5/8 : RUN apk add g++
 ---> Running in 49c35f902b2d
fetch http://dl-cdn.alpinelinux.org/alpine/v3.11/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.11/community/x86_64/APKINDEX.tar.gz
(1/13) Installing libgcc (9.2.0-r4)
(2/13) Installing libstdc++ (9.2.0-r4)
(3/13) Installing binutils (2.33.1-r0)
(4/13) Installing gmp (6.1.2-r1)
(5/13) Installing isl (0.18-r0)
(6/13) Installing libgomp (9.2.0-r4)
(7/13) Installing libatomic (9.2.0-r4)
(8/13) Installing mpfr4 (4.0.2-r1)
(9/13) Installing mpc1 (1.1.0-r1)
(10/13) Installing gcc (9.2.0-r4)
(11/13) Installing musl-dev (1.1.24-r2)
(12/13) Installing libc-dev (0.7.2-r0)
(13/13) Installing g++ (9.2.0-r4)
Executing busybox-1.31.1-r9.trigger
OK: 177 Mi

### push image

In [23]:
print("sudo docker push " + AZUREDOCKER_USER + ".azurecr.io/" + PROJNAME)

sudo docker push container2804206.azurecr.io/dash-azure


In [24]:
# upload the container image
! sudo docker push $AZUREDOCKER_USER'.azurecr.io/'$PROJNAME

The push refers to repository [container2804206.azurecr.io/dash-azure]

e5494b82: Preparing 
40269b55: Preparing 
63611e10: Preparing 
2c0b805b: Preparing 
a45a13cc: Preparing 
9655bf6d: Preparing 
6063a67d: Preparing 
b76feca4: Preparing 
40269b55: Pushed     172MB/170.6MBlatest: digest: sha256:9e0473fe716d984cd3e180d3d9949e15d10830c276b0b18342a841a40574104b size: 2206


### deploy web app
update the parameters and run the cell to generate a deployment script  
connect to cloud shell, upload, make executable and run

In [25]:
# parameters
print('RESOURCE_GROUP="' + RESOURCE_GROUP + '"')
print('LOCATION="' + LOCATION + '"')
print('APPSERVICEPLAN_NAME="' + APPSERVICEPLAN_NAME + '"')
print('APPSERVICEPLAN_COMPUTE="' + APPSERVICEPLAN_COMPUTE + '"')
print('WEBAPP_NAME="' + WEBAPP_NAME + '"')
print('AZURE_CONTAINER_REGISTRY_NAME="' + AZURE_CONTAINER_REGISTRY_NAME + '"')
print('WEBAPP_PORT="' + WEBAPP_PORT + '"')

RESOURCE_GROUP="myResourceGroup6"
LOCATION="northeurope"
APPSERVICEPLAN_NAME="myAppServicePlan6"
APPSERVICEPLAN_COMPUTE="B1"
WEBAPP_NAME="kevin2404206"
AZURE_CONTAINER_REGISTRY_NAME="container2804206.azurecr.io/dash-azure:latest"
WEBAPP_PORT="80"


In [26]:
%%writefile $project_folder/privateDashboard_deploy.azcli
#!/bin/bash

# update these parameters
RESOURCE_GROUP="myResourceGroup6"
LOCATION="northeurope"
APPSERVICEPLAN_NAME="myAppServicePlan6"
APPSERVICEPLAN_COMPUTE="B1"
WEBAPP_NAME="kevin2404206"
AZURE_CONTAINER_REGISTRY_NAME="container2804206.azurecr.io/dash-azure:latest"
WEBAPP_PORT="80"

az group create --name $RESOURCE_GROUP --location $LOCATION

az appservice plan create \
  --name $APPSERVICEPLAN_NAME \
  --resource-group $RESOURCE_GROUP \
  --is-linux \
  --location $LOCATION \
  --sku $APPSERVICEPLAN_COMPUTE

az webapp create \
  --name $WEBAPP_NAME \
  --resource-group $RESOURCE_GROUP \
  --plan $APPSERVICEPLAN_NAME \
  --deployment-container-image-name $AZURE_CONTAINER_REGISTRY_NAME

az webapp config appsettings set \
  --name $WEBAPP_NAME \
  --resource-group $RESOURCE_GROUP \
  --settings \
      PORT=$WEBAPP_PORT

# web app URL
echo "services loading... (~2min)"
echo "http://${WEBAPP_NAME}.azurewebsites.net"

az webapp log tail --name $WEBAPP_NAME --resource-group $RESOURCE_GROUP

Writing ../dashboards/dash-azure/privateDashboard_deploy.azcli


# Cleanup
remove docker images and login credentials

In [27]:
# remove all local docker image(s)
! sudo docker image prune -f -a

Deleted Images:
untagged: beire2018/dash-azure:latest
untagged: beire2018/dash-azure@sha256:6733c907d50bc8b40d50b980b0dc50a8ae9d1fac3d42cc7c7ec5aed82ca1ed98
untagged: dash-azure:latest
deleted: sha256:7d39d3496fe1cdd018f751423ccff8d9106bcb0581a604660f037746639c18ba
deleted: sha256:481d3f2576f980e0382ba51f1750443754954ddc210a61c7c39ebcae5069303c
deleted: sha256:d76fdbab75c9dafecf90de2d5f4bc5fb379fcff20f76cc315bbaa9a97b894c2b
deleted: sha256:889cb0249417902ec9960ef9ae1fc7d88a3ee36a0a4e99c3f3619fc3bed8a2f4
deleted: sha256:eda5da76a9252133976436e95a5ac39608f33468f0d1d34b2092d057ad7637a4
deleted: sha256:d7d0a7538110e7b05fc584793c780db5e09a8fd078f7effbeef7ffd51073836a
deleted: sha256:65846955f5349232334afc7eed2e1f69116442dbf56dec9bec36b90502478eaf
deleted: sha256:309765812719bc4a960283d2fced5e7dd3e3d2afbc81f51035bd160a9a44d2fc
untagged: container2804206.azurecr.io/dash-azure:latest
untagged: container2804206.azurecr.io/dash-azure@sha256:9e0473fe716d984cd3e180d3d9949e15d10830c276b0b18342a841a

In [28]:
# remove docker login file
! sudo rm ~/.docker/config.json

In [ ]:
# delete all local files that were created ?!

In [ ]:
# remove the azure container registry, save costs ?!